In [10]:
import requests
from requests import Request, Response
from bs4 import BeautifulSoup
from bs4.element import Tag
import re

In [11]:
def get_posts_url(soup: BeautifulSoup):
    # 抓取貼文列表
    links_data = soup.select('a') 
    links_data = list(filter(lambda x : x.get('class') == None, links_data))
    titles = list(filter(lambda x : x['href'].endswith('html'), links_data))
    title_relate_urls = list(map(lambda tag : tag['href'], titles))
    return title_relate_urls

# 取得貼文內容
def get_post(soup: BeautifulSoup):
    post = soup.select('meta')
    post_result = list(filter(lambda tag : tag.get("property") != None and ("title" in tag.get("property") or "description" in tag.get("property")), post))
    post_result = {k.get('property'): k.get('content') for k in post_result}
    return post_result
# 取得推文內容
def get_pushes(soup: BeautifulSoup):
    # 抓取推文內容
    pushes_element: list[Tag] = soup.find_all(["div"], attrs={"class" : "push"})
    pushes_element = list(map(lambda push_element : push_element. find_all(['span']), pushes_element))
    pushes_result = list(map(lambda push_element : {tag.get('class')[-1]: tag.get_text() for tag in push_element}, pushes_element))
    return pushes_result

# 取得文章留言
def get_post_with_pushes(soup: BeautifulSoup):
    post = get_post(soup=soup)
    post['pushes'] = get_pushes(soup=soup)
    return post

In [12]:
num_of_pages = 10

In [13]:
web_root = "https://www.ptt.cc"
group = "/bbs/Gossiping"
current_session = requests.Session()
pass_data = {
    'from':'/bbs/Gossiping/index.html',
    'yes': 'yes'
}
r_pass: Response = current_session.post("https://www.ptt.cc/ask/over18", data=pass_data)

In [14]:
# 找尋上頁
soup = BeautifulSoup(r_pass.content, 'html.parser')
tag : Tag = soup.find('a', attrs={"class" : "btn wide"}, string = re.compile("上頁"))
current_url = tag['href']

In [15]:
posts_urls = get_posts_url(soup)

In [16]:
for _ in range(10, 1, -1):
    target_url = f"{web_root}{current_url}"
    r_page = current_session.get(target_url)
    soup = BeautifulSoup(r_page.content, 'html.parser')
    posts_urls += get_posts_url(soup)
    tag : Tag = soup.find('a', attrs={"class" : "btn wide"}, string = re.compile("上頁"))
    current_url = tag['href']

In [19]:
# 取得所有貼文的網址
pre_fetch_urls = list(map(lambda path : f"{web_root}{path}", posts_urls))

In [21]:
pre_fetch_urls

['https://www.ptt.cc/bbs/Gossiping/M.1706970084.A.D36.html',
 'https://www.ptt.cc/bbs/Gossiping/M.1706970106.A.05F.html',
 'https://www.ptt.cc/bbs/Gossiping/M.1706970307.A.DF0.html',
 'https://www.ptt.cc/bbs/Gossiping/M.1706970419.A.C70.html',
 'https://www.ptt.cc/bbs/Gossiping/M.1706970545.A.1D2.html',
 'https://www.ptt.cc/bbs/Gossiping/M.1706970549.A.B6D.html',
 'https://www.ptt.cc/bbs/Gossiping/M.1699632792.A.2CB.html',
 'https://www.ptt.cc/bbs/Gossiping/M.1705931902.A.098.html',
 'https://www.ptt.cc/bbs/Gossiping/M.1706766460.A.35C.html',
 'https://www.ptt.cc/bbs/Gossiping/M.1706959211.A.063.html',
 'https://www.ptt.cc/bbs/Gossiping/M.1706969014.A.EDD.html',
 'https://www.ptt.cc/bbs/Gossiping/M.1706969157.A.00C.html',
 'https://www.ptt.cc/bbs/Gossiping/M.1706969215.A.033.html',
 'https://www.ptt.cc/bbs/Gossiping/M.1706969249.A.070.html',
 'https://www.ptt.cc/bbs/Gossiping/M.1706969399.A.7B4.html',
 'https://www.ptt.cc/bbs/Gossiping/M.1706969426.A.453.html',
 'https://www.ptt.cc/bbs

In [22]:
# 開始抓取資料
html_soups = list(map(lambda target_url : BeautifulSoup(current_session.get(target_url).content, 'html.parser'), pre_fetch_urls))

In [23]:
# 轉換資料為Json格式
results = list(map(lambda soup : get_post_with_pushes(soup=soup), html_soups))

In [26]:
# 抓取結果
print(f"Post Count = {len(results)}")

Post Count = 187


In [28]:
# 儲存為檔案
import json

In [32]:
with open('data.json', "w", encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False)